<a href="https://colab.research.google.com/github/Kira1108/gpt_sms/blob/main/prompt_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from IPython.display import clear_output
!pip install langchain openai tiktoken
clear_output()

In [30]:
import os
import openai
import json
from pprint import pprint
os.environ['OPENAI_API_KEY'] = 
openai.api_key = os.getenv("OPENAI_API_KEY")

In [13]:
import pandas as pd

data = pd.read_feather("/content/drive/MyDrive/AI/MessageGPT/data/ng_phone_msg_concat.fth")

In [24]:
from langchain.prompts import PromptTemplate

KEYWORDS_V2_TEMPLATE = """
I want you to act as a SMS service analyst, you classify message senders into categories and extract named entities from messages.
The message is a concatenation of serveral short messages sent by the same sender.
Your first task is to perform a multi-label classification.(primary_category and secondary_category)
The primary_category focus on the functionality of the message, and the secondary_category focus on the content type(industry, business type, app type) of the message.

The Primary category can be A LIST chosen from following(multiple choices):
[Advertisement, Notification, Verification, Subscription, Transaction, Reminder, Alert, Survey, Support, Invitation, Personal, Spam]

The Secondary category can be ONE OF the following classes:
[Entertainment, Banking and Finance, Retail, Telecom, Travel and Hospitality, Government and Public Services, Healthcare, Education, Social Networking, Utilities, News and Media, Non-Profit, Technology, Automotive, Food and Dining, Sports, Fashion and Beauty, Real Estate, Legal and Insurance, Job and Recruitment, Loan Service]

Your second task is to extract sender name from the message, such as a company name, a name of a mobile app, a product of a bank, loan product etc. 

Your third task it to extract keywords, a list contains at most 5 keywords that are helpful to determined the categories mentioned above.

You should format your answer in JSON FORMAT with keys being primary_category[list], secondary_category[string], sender[string] and keywords[list]
The message to analyze is delimited with triple backticks

message = ```{message}```
"""

KEYWORDS_V2_PROMPT = PromptTemplate(input_variables=["message"], template=KEYWORDS_V2_TEMPLATE)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0]['message']['content']

def parse_message(message):
    return get_completion(KEYWORDS_V2_PROMPT.format(message = message))

In [33]:
example = data.sample(1).iloc[0]

message = f"Send From: {example.phone}.\nMessage:{example.message}"

result = json.loads(parse_message(message))

print("Message: ")
print(message)

print("\n\nResult: ")
pprint(result)